$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$
# Digital Quantum Simulations

In this tutorial, we will introduce time evolution of quantum states and how to simulate them using the Trotter-Suzuki formalism.  We then show to how to run the time evolution of a one dimensional Fermi-Hubbard chain with three sites.

## Contents
1. [Introduction](#introduction)
2. [Trotter-Suzuki Formalism](#ts-equation)
3. [The Fermi-Hubbard Model](#hubbard-models)
3. [Jordan-Wigner Transformation and Encoding](#jw-transform)
4. [Qiskit Implementation](#1d-chain)

## 1. Introduction <a id='introduction'></a>

Classical simulation of physical systems typically begin by solving simple differential equations such as $\frac{dy}{dt} = f(y)$ which, to first order, has the solution $y(t+\Delta t)\approx y(t)+f(y)\Delta t$.  Meanwhile in quantum mechanics, we are concerned with the solution to $\frac{id\ket{\psi}}{dt}=H\ket{\psi}$, which for a time-indepent $H$, is
$$ \ket{\Psi(t)} = e^{iHt}\ket{\Psi(0)} $$

and simulating this requires the ability to perform the operation above.  In most cases, this reduces to matrix exponentiation, but proves to be very difficult on classical computers due to the exponential growth of the operator as the system size increases.  A good attempt at a first order solution is $\ket{\psi(t+\Delta t)}\approx (I-iH\Delta t)\ket{\psi(t)}$, but this is generally not satisfactory.  However, if we can exponentiate the Hamiltonian, we only need a sufficiently small time step to simulate our system

$$ \ket{\psi(t+\Delta t)}= e^{-iH\Delta t}\ket{\psi(t)} $$

Efficient approximations to the solution of this equation are possible for Hamiltonians which carry local interactions between the system's consitutent subsystems.  These classes of Hamiltonians can be written in the form

$$ H = \sum_k^L H_k $$

where each $H_k$ acts locally on only a portion of the total system.  For example, the terms are often just two-body interactions (such as $X_iX_j$) and one-body Hamiltonians (such as $Z_i$).  For the physicists out there, both the Ising and Hubbard models (as we'll see later on) can be written in this way.  The advantage of writing the Hamiltonian in this way is that although the total time evolution operator $e^{iHt}$ may be difficult to compute, it is much easier to use $e^{iH_kt}$ as it acts on a much smaller subsytem and is straightforward to approximate using quantum circuits.

## 2. Trotter-Suzuki Formalism <a id='ts-equation'></a>

Now armed with this idea of considering the Hamiltonian as a sum over a set of smaller subsystems, we can examine what our time evolution operator looks like.  It is important to note that because the subsystem terms $H_k$ do not generally commute with each other, we can't assume the exponential of the full Hamiltonian is a product of each subsystem (i.e. $e^{-iHt}\neq \prod_k e^{-iH_kt}$ ).

We can move past this by considering the Trotter-Suzuki formula which asymptotically approximates the sum

$$
   e^{-iHt} = e^{-i\sum_l H_lt} = \lim_{n\rightarrow\infty} \left(\prod_l e^{-iH_lt/n} \right)^n.
$$

So although we cannot exponentiate this Hamiltonian exactly, we can approximate it arbitrarily well using slices of size $1/n$.  Errors aside, this is still advantageous since we are able to efficiently implement a "difficult" gate $U(t)=e^{-iHt}$ by breaking the original problem into smaller pieces $e^{-iH_lt/n}$ which require only a limited set of elementary gates.


It may also help to provide a bit of an intuitive picture of the ST decomposition.  In his original paper discussing universal quantum simulators, Lloyd (S. Lloyd, Science 273, 1073) gave this example on the Trotter-Suzuki formalism:

<blockquote> The method for performing the simulation is conceptually straightforward, if mathematically involved. The goal is to get the simulator from point A to point B along a particular route. But the simulator can only be driven in certain directions--the operations that can be applied experimentally are limited--so it is usually not possible to go from point A to point B directly. But by moving the simulator first a little bit in one direction, then a little bit in another, then a little bit in another, and so on, it is possible to move from A to B. A car can only be driven forward and backward--it cannot be driven sideways. But it is still possible to parallel park. The following construction demonstrates a quantum analog of a familiar classical fact: By going forward and backing up a sufficiently small distance a large enough number of times, it is possible to parallel park in a space only $\epsilon$ longer than the length of the care </blockquote>

## 3. The Fermi-Hubard Model <a id='hubbard-models'></a>

Now that we have the tools available to simulate a quantum system using a quantum computer, let's choose a system to try this out on.  The Hubbard model of many electron systems was initially introduced to provide a model to consider the behavior of correlated electrons in solids, and research on it has since grown dramatically and become a mainstay within theoretical condensed matter physics.  The model itself is an extension of the tight-binding model--a simple toy model wherein electrons are allowed to "hop" between neighboring lattice sites, but do not interact with each other--and introduced an interaction term $U$ for each pair of electrons occupying the same lattice site as a means to represent Coulomb repulsion.
    
Though the Hubbard model may appear simple, this simplicity is deceptive as it turns out to be a mathematically difficult problem to solve due to its many-body nature.  Thus far an exact solution has only been found for the one-dimensional case.  Numerical simulations of the model in higher dimensions are quite common, but a true analytic solution has yet to be found.  As such, since quantum computing promises to solve many different numerically difficult problems, its incorporation into quantum computing may provide insights into these interesting systems.  Regardless, studying this model remains a staple within the condensed matter community primarily because it has emerged as a candidate model to explain high-temperature superconductivity and is a first step in understanding highly correlated physics.  

To construct the Hamiltonian of our Hubbard system, we envision a chain or lattice of sites which have space to hold up to two electrons which can either be spin up $\uparrow$ or down $\downarrow$, therefore each site can be in one of four states: $\ket{\psi} = \ket{0}, \ket{\uparrow}, \ket{\downarrow}, \ket{\uparrow \downarrow}$.  To start, let's thinking about what the kinetic energy should look like.  The simplest way to consider this is by imagining an electron "hopping" from one site to the next and is written as:
\begin{equation}
    \hat{H}_T = -t\sum_{<i,j>,\sigma}(\hat{c}_{i,\sigma}^\dagger\hat{c}_{j,\sigma} + \hat{c}_{j,\sigma}^\dagger\hat{c}_{i,\sigma})
\end{equation}
where the sum is over the spin, $\sigma$, and nearest neighbor pairs, $<i,j>$, and $t$ is the energy scale governing the hopping between neighbors and determined by the overlap of the two wavefunctions.  The operators $\hat{c}$ and $\hat{c}^\dagger$ are known as the "creation" and "annihilation" operators and provide a useful tool for us which first appear in most textbooks in the quantum harmonic oscillator system (the details of which can be left aside for now).  The takeaway here is that the creation and annihilation operators will add and remove occupations at a given site $i$ with spin $\sigma$ and return the value of the occupation at that site

\begin{equation}
    \hat{c}^\dagger\ket{n} = \sqrt{n+1}\ket{n+1} \ \ \ \ 
    \hat{c}\ket{n} = \sqrt{n}\ket{n-1}
\end{equation}
with the stipulation that applying the lowering operator to the vacuum state is $\hat{c}\ket{0}=0$.


We can use these operators in a lattice of atomic sites which can host electrons in a single orbital with either spin $\uparrow$ or $\downarrow$ and can be singly ($\ket{\uparrow}$, $\ket{\downarrow}$), doubly ($\ket{\uparrow\downarrow}$), or completely unoccupied ($\ket{0})$.  In this system, we describe the state basis as a direct product for each of the site occupations $\ket{\psi}=\ket{n_1,...,n_N}$.  The ladder operators become creation and annihilation operators at a site $i$ with spin $\sigma$ ($\hat{c}^\dagger_{i,\sigma}$ and $\hat{c}_{i,\sigma}$ respectively).  It is also important to note that in this picture, since the operators describe fermions, they are defined by the anticommutation relations such that
\begin{align}
    \{\hat{c}_{i,\sigma}^\dagger,\hat{c}_{j,\sigma'}\} &= \delta_{i,j}\delta_{\sigma,\sigma'} \\
    \{\hat{c}_{i,\sigma}^\dagger,\hat{c}_{i,\sigma'}^\dagger\}& = 0 \\
    \{\hat{c}_{i,\sigma},\hat{c}_{j,\sigma'}\} &= 0.
\end{align}


There is also a number operator defined as $\hat{n}=\hat{c}^\dagger\hat{c}$ which "counts" the excited states.  The Hubbard model crudely accounts for electron interaction via a screened Coulomb interaction.  In this model we approximate this by considering that the largest interaction will occur for two electrons occupying the same site and zero otherwise.  This has the form
\begin{equation}
    \hat{H}_U = U\sum_i\hat{n}_{i\uparrow}\hat{n}_{i\downarrow}
\end{equation}
where $\hat{n}_{i,\sigma}=\hat{c}_{i,\sigma}^\dagger\hat{c}_{i,\sigma}$ is the number operator as before.  Lastly, we also should keep track of how many electrons are in the system.  Here we can introduce a chemical potential $\mu$ which acts as the energy added to the system when an electron is introduced into the system and is related to the total number of electrons in the system
\begin{equation}
    \hat{H}_\mu = \mu\sum_{i,\sigma}\hat{n}_{i,\sigma}.
\end{equation}
Together, dropping all the hats, the Hamiltonian has the form
\begin{equation}
    \boxed{H =  -t\sum_{<i,j>,\sigma}(c_{i,\sigma}^\dagger c_{j,\sigma} + c_{j,\sigma}^\dagger c_{i,\sigma}) + U\sum_in_{i\uparrow}n_{i\downarrow} - \mu\sum_{i,\sigma}n_{i,\sigma}}.
\end{equation}


A simple example of this model is to consider a system of two sites.  We can write the state in the occupation number basis as $\ket{\psi}=\ket{n_{1\uparrow}, n_{1\downarrow}}\ket{n_{2\uparrow},n_{2\downarrow}}$ where $n_{i,\sigma}$ can be either $0$ or $1$.  Out of all the $2^4=16$ possible states, we can separate these into three sets of states depending on the kinetic energy (or how many electrons are allowed to hop between sites).  The first set contains the completely empty lattice, the completely filled lattice, and the states with two electrons with the same spin since each of these have zero kinetic energy since hopping is not allowed or there are no electrons present to hop.  The second set are those states which allow one electron to hop between sites, either the singly or triply occupied system.  The third set are the two states which allow two electrons to hop between the sites.


## 4. Jordan-Wigner Transformation and Encoding <a id='jw-transform'></a>
To utilize quantum computing when simulating Hubbard-like systems, one needs to first transform the Hubbard Hamiltonian into a set of operations accessible to quantum computers.  Namely, we use the Jordan-Wigner transformation to map the creation and annihilation operators of the Hubbard Hamiltonian to a set of Pauli matrices.  Traditionally, the Hubbard Hamiltonian in second quantization utilize creation and annihilation operators $\hat{c}_{j,\sigma}$ and $\hat{c}_{j,\sigma}^\dagger$ which changes the occupation of a given spin orbital, as described above.  In a quantum computing environment, we map each fermionic mode (a given site with a given spin) to a single qubit.  Since we are now indexing by mode, we then drop the spin index and sum over the fermionic modes with the action of each operator being $\hat{c}_{j}^\dagger\ket{0}_{j} = \ket{1}_{j}$,  $\hat{c}_{j}\ket{1}_{j} = \ket{0}_{j}$, $\hat{c}^\dagger_{j}\ket{1}_{j}=0$, and $\hat{c}_{j}\ket{0}_{j}=0$ for each fermionic mode $j$.  It is straightforward to write their matrix representation in terms of the Pauli matrices
\begin{align}
    \hat{c}_{j}^\dagger = \begin{pmatrix} 0 & 0 \\ 1 & 0 \end{pmatrix} = \frac{X_{j} - iY_{j}}{2} \nonumber \\
    \hat{c}_{j} = \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix} = \frac{X_{j} + iY_{j}}{2}
    \label{eq:c-cdagger}
\end{align}
where $X_{j}$, $Y_{j}$ are the associated Pauli operators acting on the j$^{th}$ mode.  However, this mapping does not account for the antisymmetric characteristics of fermions.  We know that $\hat{c}_{j}$ and $\hat{c}_{j}^\dagger$, as defined in the previous section, anti-commute but this combination of Pauli matrices do not.  This can be remedied by noting that the Pauli matrices naturally anti-commute by $XZ = -ZX$ and $YZ = -ZY$.  Interspersing $Z$ operators into the construction will thus emulate the anti-commutation needed which is shown as follows

\begin{align}
    \hat{c}_{1}^\dagger &= \left(\frac{X_1-iY_1}{2}\right)\otimes 1 \otimes 1 \otimes ... \otimes 1  \nonumber \\
    \hat{c}_{2}^\dagger &= Z_1 \otimes \left(\frac{X_2-iY_2}{2}\right)\otimes 1 \otimes 1 \otimes ... \otimes 1 \nonumber \\
    \hat{c}_{3}^\dagger &= Z_1 \otimes Z_2 \otimes \left(\frac{X_3-iY_3}{2}\right)\otimes 1 \otimes 1 \otimes ... \otimes 1 \nonumber \\
    &\vdots \nonumber \\
    \hat{c}_{N}^\dagger &= Z_1\otimes Z_2\otimes Z_3\otimes ... \otimes \left(\frac{X_N-iY_N}{2}\right).
\end{align}

We can also rewrite the number operator as 
\begin{equation} 
    \hat{n}_{j} = \hat{c}_{j}^\dagger \hat{c}_{j} = \frac{(1-Z_{j})}{2}
\end{equation}
and put these together into the Hubbard Hamiltonian to obtain
\begin{equation}
  \boxed{  H = -\frac{t}{2}\sum_{<ij>}Z_{j+1:i-1}(X_{i}X_{j} + Y_{i}Y_{j}) + \frac{U}{4}\sum_i(I-Z_{i}^{\uparrow})(I-Z_{i}^{\downarrow}) }
\end{equation}
where $Z_{j+1:i-1}$ is the matrix product of $Z$ operators from mode $j+1$ to mode $i-1$ (typically there is only one $Z$ if the sum is over nearest neighbor sites).    

## 5. Qiskit Implementation <a id='1d-chain'></a>
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

Let's now take a relatively simple example and apply it to a quantum computer.  We will simulate the time evolution of the Hubbard Hamiltonian for a one-dimensional chain of three sites:

\begin{equation} H = -t\sum_{<ij>,\sigma}(\hat{c}_{i_\sigma}\hat{c}_{j_\sigma} + \hat{c}_{j_\sigma}^\dagger\hat{c}_{i_\sigma} ) + U\sum_i\hat{c}_{i_\uparrow}^{\dagger}\hat{c}_{i_\uparrow}\hat{c}_{i_\downarrow}^{\dagger}\hat{c}_{i_\downarrow}
\end{equation}
which under the Jordan-Wigner mapping,becomes:

\begin{equation}
H = -\frac{t}{2}\sum_{<ij>}Z_{i+1:j-1}(X_{i}X_{j} + Y_{i}Y_{j}) + \frac{U}{4}\sum_{ij}(I-Z_{i})(I-Z_{j})
\end{equation}
where $Z_{i}$, $X_{i}$, and $Y_{i}$ are the corresponding Pauli matrices acting on the $i^{th}$ fermionic mode and for a chain with only 3-sites, there are no Pauli $Z$ strings in the hopping term of the Hamiltonian.


    
Before going further, let's import some libraries that we'll be needing later

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.quantum_info import Operator
from qiskit.tools.monitor import job_monitor
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from matplotlib import rcParams
rcParams['text.usetex'] = True

#Useful tool for converting an integer to a binary bit string
def get_bin(x, n=0):
    """
    Get the binary representation of x.
    Parameters: x (int), n (int, number of digits)"""
    return format(x, 'b').zfill(n)

### 3-Site Hamiltonian and Qubit Mapping

Writing out the sum over the three sites, the Hamiltonian becomes:

\begin{align}
     H = &-\frac{t}{2}(X_0X_1 + Y_0Y_1) - \frac{t}{2}(X_1X_2 + Y_1Y_2) \nonumber \\ 
&-\frac{t}{2}(X_3X_4 + Y_3Y_4) - \frac{t}{2}(X_4X_5 + Y_4Y_5) \nonumber\\
&+ \frac{U}{4}(I-Z_0)(I-Z_3)+ \frac{U}{4}(I-Z_1)(I-Z_4) + \frac{U}{4}(I-Z_2)(I-Z_5) \nonumber\\
= &H_{01} + H_{12} + H_{23} + H_{34} + H_{03} + H_{14} + H_{25}
\end{align}

#### Qubit Mapping

Each site in the 3-site chain is represented by two qubits, one for each spin, and the wavefunction is represented as

$$ \ket{\psi} = \ket{q_0}\ket{q_1}\ket{q_2}\ket{q_3}\ket{q_4}\ket{q_5} $$

where $\ket{q_i} = \{ \ket{0}, \ket{1} \} $ represent unoccupied or occupied fermionic modes, $i=0,1,2$ are the spin up electron occupations and $i=3,4,5$ are the spin down electron occupations.


### Time Evolution

We want to simulate the time evolution of $\ket{\psi}$ via 

$$ \ket{\psi(t+\Delta t)} = e^{-iH\Delta t}\ket{\psi(t)} $$


Which we can approximate by separating the Hamiltonian into its constituent terms over the fermionic modes and a small enough time step $\Delta t$

$$e^{iH\Delta t} \approx e^{iH_{10}^{\uparrow}\Delta t}e^{iH_{12}^{\uparrow}\Delta t}e^{iH_{10}^{\downarrow}\Delta t}e^{iH_{12}^{\downarrow}\Delta t}e^{iH_0\Delta t}e^{iH_1\Delta t}e^{iH_2\Delta t} $$.


So what do the gates look like for each of these terms?

##### Hopping Terms

For each pair of hopping terms we have
\begin{equation}e^{-i\Delta t(\frac{-t}{2})(X_iX_j + Y_iY_j)} \approx e^{\frac{it\Delta t}{2}X_iX_j} e^{\frac{it\Delta t}{2}Y_iY_j}.
\label{eq:op-expansion}
\end{equation}

Expanding the $X_iX_j$ term

\begin{align}
e^{\frac{it\Delta t}{2}X_iX_j} = & \sum_{k=0}^{\infty} \frac{1}{k!}\left(\frac{it\Delta t}{2}X_iX_j\right)^k \nonumber \\
 =& \sum_{k, even}\frac{i^k}{k!}\left( \frac{t\Delta t}{2} \right)^k I + \sum_{k, odd}\frac{i^k}{k!}\left( \frac{t\Delta t}{2} \right)X_i X_j \nonumber \\ 
 = &\cos\left(\frac{t\Delta t}{2}\right)I + i\sin\left( \frac{t\Delta t}{2}\right)X_i X_j \nonumber \\
 = &\begin{pmatrix}\cos\theta & 0 & 0 & i\sin\theta \\ 0 & \cos\theta & i\sin\theta & 0 \\ 0 & i\sin\theta & \cos\theta & 0 \\ i\sin\theta & 0 & 0 & \cos\theta\end{pmatrix},
\end{align}
with $\theta=\frac{t\Delta t}{2}$, and written in the $\ket{q_{i}q_{j}}$ basis.

Similarly for the $Y_i Y_j$ terms

\begin{align}
    e^{i\frac{t\Delta t}{2}Y_i Y_j} =& \cos\left(\frac{t\Delta t}{2}\right)I + i\sin\left(\frac{t\Delta t}{2}\right)Y_i Y_j \nonumber \\
= & \begin{pmatrix}\cos\theta & 0 & 0 & -i\sin\theta \\ 0 & \cos\theta & i\sin\theta & 0 \\ 0 & i\sin\theta & \cos\theta & 0 \\ -i\sin\theta & 0 & 0 & \cos\theta\end{pmatrix}.
\end{align}


Note also that these matrices are diagonal save for the 4x4 block corresponding to a gate acting on qubits $i$ and $j$.

#### On-Site Terms


Now we'll expand the on-site term, $e^{i\frac{U\Delta t}{4}(I-Z_i)(I-Z_j)}$.  First we examine the powers of $(I-Z_i)(I-Z_j)$:

\begin{align} (I-Z_i)^2(I-Z_j)^2 &= (I + I - 2Z_i)(I+I-2Z_j) = 4(I-Z_i)(I-Z_j) \\
(I-Z_i)^3(I-Z_j)^3& = (I-Z_i)(I-Z_j)(I-Z_i)^2(I-Z_j)^2 \nonumber \\
&= 4(I-Z_i)^2(I-Z_j)^2 = 16(I-Z_i)(I-Z_j)\\
(I-Z_i)^4(I-Z_j)^4& = 16(I-Z_i)^2(I-Z_j)^2 = 4^3(I-Z_i)(I-Z_j)\\
\implies (I-Z_i)^k(I-Z_j)^k& = 4^{k-1}(I-Z_i)(I-Z_j), \end{align}

then writing out the expansion of $e^{i\frac{U\Delta t}{4}(I-Z_i)(I-Z_j)}$ we get

\begin{align} e^{i\frac{U\Delta t}{4}(I-Z_i)(I-Z_j)} &= \sum_k \frac{1}{k!}\left(\frac{i\Delta tU}{4}\right)^k(I-Z_i)^k(I-Z_j)^k \\
&= I+(I-Z_i)(I-Z_j)\sum_k \frac{\left(i\Delta tU\right)^k}{k!}\frac{4^{k-1}}{4^k} - \frac{1}{4}(I-Z_i)(I-Z_j) \\
& = I-\frac{1}{4}(I-Z_i)(I-Z_j) + \frac{1}{4}e^{iU\Delta t}(I-Z_i)(I-Z_j)  \\
& \boxed{= I-(I-Z_i)(I-Z_j)\left(1-e^{iU\Delta t} \right) } \\
&= \begin{pmatrix}1 & 0 & 0 & 0\\ 0 & 1& 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & e^{iU\Delta t} \end{pmatrix} .
\end{align}


$^1$https://web.cs.ucdavis.edu/~bai/publications/varneyleebai09.pdf

### References

@article{hubbard-paper,
 ISSN = {00804630},
 URL = {http://www.jstor.org/stable/2414761},
 author = {Hubbard, J.},
 journal = {Proceedings of the Royal Society of London. Series A, Mathematical and Physical Sciences},
 number = {1365},
 pages = {238--257},
 publisher = {The Royal Society},
 title = {Electron Correlations in Narrow Energy Bands},
 volume = {276},
 year = {1963}
}

@article{kanamori-hubbard,
    author = {Kanamori, Junjiro},
    title = "{Electron Correlation and Ferromagnetism of Transition Metals}",
    journal = {Progress of Theoretical Physics},
    volume = {30},
    number = {3},
    pages = {275-289},
    year = {1963},
    month = {09},
    issn = {0033-068X},
    doi = {10.1143/PTP.30.275},
    url = {https://doi.org/10.1143/PTP.30.275},
    eprint = {https://academic.oup.com/ptp/article-pdf/30/3/275/5278869/30-3-275.pdf},
}


@article{gutzwiller-hubbard,
  title = {Effect of Correlation on the Ferromagnetism of Transition Metals},
  author = {Gutzwiller, Martin C.},
  journal = {Phys. Rev. Lett.},
  volume = {10},
  issue = {5},
  pages = {159--162},
  numpages = {0},
  year = {1963},
  month = {Mar},
  publisher = {American Physical Society},
  doi = {10.1103/PhysRevLett.10.159},
  url = {https://link.aps.org/doi/10.1103/PhysRevLett.10.159}
}


@article{Hubbard-III,
author = {Hubbard, J.  and Flowers, Brian Hilton },
title = {Electron correlations in narrow energy bands III. An improved solution},
journal = {Proceedings of the Royal Society of London. Series A. Mathematical and Physical Sciences},
volume = {281},
number = {1386},
pages = {401-419},
year = {1964},
doi = {10.1098/rspa.1964.0190},
URL = {https://royalsocietypublishing.org/doi/abs/10.1098/rspa.1964.0190},
eprint = {https://royalsocietypublishing.org/doi/pdf/10.1098/rspa.1964.0190}
}

@article{1d-hubbard,
  title = {Absence of Mott Transition in an Exact Solution of the Short-Range, One-Band Model in One Dimension},
  author = {Lieb, Elliott H. and Wu, F. Y.},
  journal = {Phys. Rev. Lett.},
  volume = {20},
  issue = {25},
  pages = {1445--1448},
  numpages = {0},
  year = {1968},
  month = {Jun},
  publisher = {American Physical Society},
  doi = {10.1103/PhysRevLett.20.1445},
  url = {https://link.aps.org/doi/10.1103/PhysRevLett.20.1445}
}